In [36]:
# pip install scikit-learn

In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch
import numpy as np
from sklearn.metrics import classification_report, hamming_loss, accuracy_score
from sklearn.metrics import f1_score


In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
df = pd.read_csv("/content/drive/MyDrive/chexpert_data_v2/valid.csv", sep=",")
image_embedding_path = "/content/drive/MyDrive/Embedings/Image_embeding/image_embeddings_LRF30%.pt"
text_embedding_path = "/content/drive/MyDrive/Embedings/Text_embeding/disease_text_embeddings.pt"

In [41]:
image_embeddings = torch.load(image_embedding_path)
text_embeddings = torch.load(text_embedding_path)
disease_columns = list(text_embeddings.keys())

In [42]:
disease_columns

['Atelectasis',
 'Cardiomegaly',
 'Consolidation',
 'Edema',
 'Pleural Effusion',
 'Pneumonia',
 'Pneumothorax']

In [43]:
disease_columns = [
    'Atelectasis',
    'Cardiomegaly',
    'Consolidation',
    'Edema',
    'Pleural Effusion',
    'Pneumonia',
    'Pneumothorax'
]

In [44]:
# print(df.columns)


In [45]:
label_dict = {}
prefix = "CheXpert-v1.0-small/"

for i, row in df.iterrows():
    path = row["Path"]

    if path.startswith(prefix):
        path = path[len(prefix):]

    label = []
    for disease in disease_columns:
        val = row[disease]
        if pd.isna(val):
            label.append(0.0)
        else:
            label.append(float(val))
    label_dict[path] = torch.tensor(label)


In [46]:
img_paths = list(image_embeddings.keys())
labels = torch.stack([label_dict[p] for p in img_paths])


In [47]:
# img_paths

In [48]:
img_keys = list(image_embeddings.keys())
img_tensor = torch.stack([image_embeddings[k] for k in img_keys])  # [N, r]

disease_keys = list(text_embeddings.keys())
image_repr = img_tensor.mean(dim=1)


In [49]:
# print(img_tensor.size())
# print(text_tensor.size())


In [50]:
text_tensor = torch.stack([text_embeddings[k] for k in text_embeddings]).to(device)  # [7, 768]
text_tensor = F.normalize(text_tensor, dim=1)


In [51]:
proj = torch.nn.Linear(64, 768).to(device)
image_proj = proj(image_repr)  # [234, 768]
image_proj = F.normalize(image_proj, dim=1)


In [52]:
similarity = image_proj @ text_tensor.T  # [234, 7]

In [53]:
print(image_repr.size())

torch.Size([234, 64])


In [54]:
probs = torch.sigmoid(similarity)  # [234, 7]
threshold = 0.5
preds = (probs > threshold).float()


In [55]:
accuracy = (preds == labels).all(dim=1).float().mean()
print(f"Exact match accuracy: {accuracy.item():.4f}")


sample_accuracy = (preds == labels).float().mean()
print(f"Sample-wise accuracy: {sample_accuracy.item():.4f}")

per_label_acc = ((preds == labels).float().mean(dim=0))
for i, disease in enumerate(disease_columns):
    print(f"{disease}: {per_label_acc[i].item():.4f}")

macro_accuracy = per_label_acc.mean()
print(f"Macro (mean per-label) accuracy: {macro_accuracy.item():.4f}")


Exact match accuracy: 0.3547
Sample-wise accuracy: 0.7576
Atelectasis: 0.6923
Cardiomegaly: 0.6880
Consolidation: 0.8205
Edema: 0.7051
Pleural Effusion: 0.7607
Pneumonia: 0.7607
Pneumothorax: 0.8761
Macro (mean per-label) accuracy: 0.7576


In [56]:
y_true = labels.cpu().numpy()
y_pred = preds.cpu().numpy()

target_names = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
                'Pleural Effusion', 'Pneumonia', 'Pneumothorax']

print("🔍 Classification Report (Macro & Per-label):")
print(classification_report(y_true, y_pred, target_names=target_names, zero_division=0))

#  Hamming Loss
hl = hamming_loss(y_true, y_pred)
print(f"\n Hamming Loss: {hl:.4f}")

#  Exact Match Ratio (sample-wise accuracy)
exact_match = np.all(y_true == y_pred, axis=1).mean()
print(f" Exact Match Accuracy: {exact_match:.4f}")


🔍 Classification Report (Macro & Per-label):
                  precision    recall  f1-score   support

     Atelectasis       0.79      0.14      0.23        80
    Cardiomegaly       0.46      0.49      0.47        68
   Consolidation       0.33      0.27      0.30        33
           Edema       0.25      0.27      0.26        45
Pleural Effusion       0.76      0.24      0.36        67
       Pneumonia       0.06      0.38      0.10         8
    Pneumothorax       0.04      0.12      0.06         8

       micro avg       0.33      0.28      0.30       309
       macro avg       0.38      0.27      0.26       309
    weighted avg       0.55      0.28      0.32       309
     samples avg       0.10      0.13      0.10       309


 Hamming Loss: 0.2424
 Exact Match Accuracy: 0.3547


In [57]:
best_threshold = 0.0
best_f1 = 0.0

for t in np.arange(0.1, 0.9, 0.05):
    preds = (probs > t).int()
    f1 = f1_score(y_true, preds.numpy(), average='macro', zero_division=0)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

print(f"  threshold: {best_threshold:.2f} با F1: {best_f1:.4f}")


  threshold: 0.10 با F1: 0.3010


In [58]:
loss = F.binary_cross_entropy_with_logits(similarity, labels.float())


In [59]:
text_tensor.shape

torch.Size([7, 768])

In [60]:
labels.shape

torch.Size([234, 7])

In [61]:
image_proj.shape

torch.Size([234, 768])